In [2]:
import pandas as pd #Dataframe, Series
import numpy as np

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
import seaborn as sns

import graphviz
import pydotplus
import io

from scipy import misc

from sklearn.metrics import accuracy_score

import spotipy
import spotipy.util as util
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 

In [3]:
cid ="0cadd882a6ab4ff485c80b8b02aa3b0c" 
secret = "04d0f737e18a4a92abee1da25d70766b"
username = ""

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

scope = 'user-library-read playlist-read-private'
token = util.prompt_for_user_token(username, scope)

if token:
    sp = spotipy.Spotify(auth=token)
else:
    print("Can't get token for", username)

In [8]:
data = []
tracks = sp.current_user_saved_tracks()
while tracks["next"]:
    for track in tracks["items"]:
        newObject = {}
        newObject["added_at"] = track["added_at"]
        newObject["trackName"] = track["track"]["name"]
        newObject["id"] = track["track"]["id"]
        data.append(newObject)
    tracks = sp.next(tracks)

[{'added_at': '2018-02-22T07:04:10Z',
  'id': '7ej7R4LjMWP6sdCtoayJSN',
  'trackName': 'The River'},
 {'added_at': '2018-02-19T19:23:33Z',
  'id': '57waeauCsihehcGXgZQMar',
  'trackName': 'Streetlights on Mars'},
 {'added_at': '2018-02-19T03:47:23Z',
  'id': '3kE3u6ZqtcNbebDOD5RcIe',
  'trackName': 'Us'},
 {'added_at': '2018-02-14T07:33:06Z',
  'id': '0AVs4KwMZncK8PVJPmff4A',
  'trackName': 'Ready Yet'},
 {'added_at': '2018-02-14T07:32:56Z',
  'id': '3bjLCKsBNSFyx6Gfsb7X4h',
  'trackName': 'love ride'},
 {'added_at': '2018-02-12T20:17:43Z',
  'id': '10O3TrugQwEBsd6l3M6zG2',
  'trackName': 'Awkward'},
 {'added_at': '2018-02-12T19:46:27Z',
  'id': '4tTzdneIuunKksESB71cws',
  'trackName': 'Play Me Like a Violin'},
 {'added_at': '2018-02-09T18:30:49Z',
  'id': '5yFSF6qQA1rcLsQRnBsZgb',
  'trackName': 'All Falls Down'},
 {'added_at': '2018-02-09T18:15:00Z',
  'id': '4hOyjY9EUksjyylV1QZaF3',
  'trackName': 'Normal'},
 {'added_at': '2018-02-09T08:19:10Z',
  'id': '7ahYQRrOQSXJ5OhsLnpfU8',
  '

In [35]:
features = []

for i in range(0,len(data)):
    audio_features = sp.audio_features(data[i]['id'])
    for track in audio_features:
        features.append(track)
        
features

[{'acousticness': 0.11,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/7ej7R4LjMWP6sdCtoayJSN',
  'danceability': 0.497,
  'duration_ms': 212211,
  'energy': 0.649,
  'id': '7ej7R4LjMWP6sdCtoayJSN',
  'instrumentalness': 0,
  'key': 0,
  'liveness': 0.385,
  'loudness': -4.994,
  'mode': 1,
  'speechiness': 0.033,
  'tempo': 95.044,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/7ej7R4LjMWP6sdCtoayJSN',
  'type': 'audio_features',
  'uri': 'spotify:track:7ej7R4LjMWP6sdCtoayJSN',
  'valence': 0.299},
 {'acousticness': 0.507,
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/57waeauCsihehcGXgZQMar',
  'danceability': 0.432,
  'duration_ms': 228267,
  'energy': 0.582,
  'id': '57waeauCsihehcGXgZQMar',
  'instrumentalness': 0.000316,
  'key': 1,
  'liveness': 0.354,
  'loudness': -10.047,
  'mode': 0,
  'speechiness': 0.194,
  'tempo': 90.991,
  'time_signature': 4,
  'track_href': 'https://api.spotify.com/v1/tracks/57waeauCsihehcGXgZQMar

In [36]:
def merge_lists(l1, l2, key):
  merged = {}
  for item in l1+l2:
    if item[key] in merged:
      merged[item[key]].update(item)
    else:
      merged[item[key]] = item
  return [val for (_, val) in merged.items()]

In [39]:
mergedData = merge_lists(data, features , 'id')

df= pd.DataFrame(mergedData)

In [61]:
# import datetime, dateutil.parser

# for data in mergedData:
#     d = dateutil.parser.parse(data["added_at"])
#     mergedData[-1]["day"] = d.day
#     mergedData[-1]["month"] = d.month
#     mergedData[-1]["year"] = d.year
#     #print(str(d.day) + " " + str(d.month) + " " + str(d.year))
    
print(mergedData[0])

{'liveness': 0.0882, 'instrumentalness': 0, 'time_signature': 4, 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/4alxY2DDLGFliiVSojsoO3', 'type': 'audio_features', 'duration_ms': 246500, 'added_at': '2016-07-07T22:45:32Z', 'valence': 0.122, 'energy': 0.557, 'track_href': 'https://api.spotify.com/v1/tracks/4alxY2DDLGFliiVSojsoO3', 'loudness': -5.643, 'id': '4alxY2DDLGFliiVSojsoO3', 'acousticness': 0.431, 'key': 3, 'tempo': 79.832, 'trackName': 'Your Life', 'speechiness': 0.0708, 'mode': 0, 'danceability': 0.458, 'uri': 'spotify:track:4alxY2DDLGFliiVSojsoO3'}


In [62]:
df.to_csv("mySongsWithDate.csv")